## Импортируем необходимые библиотеки

In [667]:
import pandas as pd
import csv

## Создаем датафрейм покупателей

In [668]:
def customercsv_to_df(path):
    with open(f'{path}', 'r') as customer:
        customer_lst = []
        
        for el in customer:
            customer_lst.append(el.split('\t'))
    
        customer_df = [dict(id=el[0],
                            name=el[1],
                            email=el[2],
                            joinDate=el[3],
                            status=el[4].strip('\n')) for el in customer_lst]
        
        return customer_df

## Создаем датафрейм товаров

In [669]:
def productcsv_to_df(path):
    with open(f'{path}', 'r') as product:
        product_lst = []
        
        for el in product:
            product_lst.append(el.split('\t'))
    
        product_df = [dict(id=el[0],
                           name=el[1],
                           price=el[2],
                           numberOfProducts=el[3].strip('\n')) for el in product_lst]
        
        return product_df

## Создаем датафрейм заказов

In [670]:
def ordercsv_to_df(path):
    with open(f'{path}', 'r') as order:
        order_lst = []
        
        for el in order:
            order_lst.append(el.split('\t'))
    
        order_df = [dict(customerID=int(el[0]),
                           orderID=int(el[1]),
                           productID=int(el[2]),
                           numberOfProduct=int(el[3]),
                           orderDate=el[4],
                           status=el[5].strip('\n')) for el in order_lst]
        
        return order_df

## Создаем функцию записи словаря в csv

In [671]:
def dict_to_csv(dict):
    with open('statistic.csv', 'w', newline='') as statisic:
        field_names = ['customer.name', 'product.name']
        writer = csv.DictWriter(statisic, fieldnames=field_names)
        writer.writeheader()
        for k, v in dict.items():
            writer.writerow({field_names[0]: k, field_names[1]: v})

## Создаем главную функцию


In [672]:
def main():
    
    path = ['Beeline_practise/customer.csv',
            'Beeline_practise/product.csv',
            'Beeline_practise/order.csv']
    
    customer_df = pd.DataFrame(customercsv_to_df(path[0]))
    product_df = pd.DataFrame(productcsv_to_df(path[1]))
    order_df = pd.DataFrame(ordercsv_to_df(path[2]))
    
    order_df = order_df.loc[order_df['customerID'] < 10]                                        #чистим данные
    order_df.insert(0, 'id_operation', [el for el in range(1, len(order_df) + 1)], False)
                                                                                                  
    result_dict = {i: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] for i in range(len(customer_df['name']))}  #Создаем словарь, ключи которого- id клиента, значения - списки возможных товаров
    
    for i, c, v in zip(order_df['id_operation'], order_df['customerID'], order_df['productID']):#заполняем списки возможных товаров
        if i == 45:
            continue
        result_dict[c-1][v-1] += order_df['numberOfProduct'][i]
    
    for k, v in result_dict.items():#меняем списки товаров на id популярного товара из списка
        max_vl = max(result_dict[k])
        if max_vl == 0:
            result_dict[k] = 'Популярный продукт отсутствует'
            continue
        result_dict[k] = result_dict[k].index(max_vl)
    
    for k, v in result_dict.items():#меняем id популярного товара на его название
        if result_dict[k] in product_df['id']:
            result_dict[k] = product_df['name'][k]
        
    for key in list(result_dict):#меняем ключи словаря на имена покупателей
        if key in customer_df['id']:
            result_dict[customer_df['name'][key]] = result_dict.pop(key)
    
    dict_to_csv(result_dict)#записываем словарь в csv файл

In [673]:
if __name__ == '__main__':
    main()